<a href="https://colab.research.google.com/github/vnsh69/machine-learning-practice/blob/main/cnn_pizza%26steak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

--2024-11-01 06:56:08--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 142.251.183.207, 173.194.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109540975 (104M) [application/zip]
Saving to: ‘pizza_steak.zip’

pizza_steak.zip     100%[===================>] 104.47M   191MB/s    in 0.5s    

2024-11-01 06:56:09 (191 MB/s) - ‘pizza_steak.zip’ saved [109540975/109540975]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random

1. preprocessing the data

In [6]:
train = "pizza_steak/train/"
test = "pizza_steak/test/"

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data = ImageDataGenerator(rescale=1/255)
test_data = ImageDataGenerator(rescale=1/255)

In [11]:
train_data = train_data.flow_from_directory(train,target_size=(224,224),batch_size=32,class_mode="binary")
test_data = test_data.flow_from_directory(test,target_size=(224,224),batch_size=32,class_mode="binary")


Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [19]:
train_data[0]

(array([[[[0.09411766, 0.0627451 , 0.10588236],
          [0.10196079, 0.07058824, 0.1137255 ],
          [0.08235294, 0.0627451 , 0.09019608],
          ...,
          [0.48235297, 0.427451  , 0.3254902 ],
          [0.50980395, 0.4666667 , 0.35686275],
          [0.49411768, 0.45098042, 0.3254902 ]],
 
         [[0.08627451, 0.05490196, 0.09803922],
          [0.0627451 , 0.03921569, 0.07843138],
          [0.04705883, 0.02745098, 0.05490196],
          ...,
          [0.5254902 , 0.454902  , 0.36078432],
          [0.5176471 , 0.4431373 , 0.3254902 ],
          [0.5294118 , 0.44705886, 0.3254902 ]],
 
         [[0.04313726, 0.01960784, 0.05882353],
          [0.04313726, 0.01960784, 0.05882353],
          [0.0509804 , 0.02745098, 0.06666667],
          ...,
          [0.38431376, 0.3137255 , 0.23529413],
          [0.454902  , 0.37647063, 0.28235295],
          [0.454902  , 0.37254903, 0.26666668]],
 
         ...,
 
         [[0.10196079, 0.08627451, 0.14509805],
          [0.     

In [20]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.models import Sequential

In [30]:
model = Sequential([
    Conv2D(
        filters=10,
        kernel_size=3,
        activation="relu",
        padding="valid",
        input_shape=(224,224,3)
    ),
    Conv2D(10,3,activation="relu"),
    Conv2D(10,3,activation="relu"),
    Flatten(),
    Dense(1,activation="sigmoid")
])

In [31]:
model.compile(loss="binary_crossentropy",optimizer=Adam(),metrics=["accuracy"])

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 10)      280       
                                                                 
 conv2d_7 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 conv2d_8 (Conv2D)           (None, 218, 218, 10)      910       
                                                                 
 flatten_2 (Flatten)         (None, 475240)            0         
                                                                 
 dense_2 (Dense)             (None, 1)                 475241    
                                                                 
Total params: 477341 (1.82 MB)
Trainable params: 477341 (1.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
history = model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data)
)

Epoch 1/5
47/47 [==============================] - 20s 420ms/step - loss: 0.6871 - accuracy: 0.6793 - val_loss: 0.3897 - val_accuracy: 0.8240
Epoch 2/5
47/47 [==============================] - 20s 414ms/step - loss: 0.3711 - accuracy: 0.8507 - val_loss: 0.4241 - val_accuracy: 0.7900
Epoch 3/5
47/47 [==============================] - 20s 415ms/step - loss: 0.2935 - accuracy: 0.8787 - val_loss: 0.3646 - val_accuracy: 0.8480
Epoch 4/5
47/47 [==============================] - 19s 410ms/step - loss: 0.1066 - accuracy: 0.9667 - val_loss: 0.4603 - val_accuracy: 0.8060
Epoch 5/5
47/47 [==============================] - 20s 414ms/step - loss: 0.0434 - accuracy: 0.9893 - val_loss: 0.4590 - val_accuracy: 0.8300
